In [2]:
task_id = "9e06dc7d106c405eb8fe3cdf55173244"

In [3]:
from clearml import Task


def get_task_artifacts(task_id: str) -> dict:
    task = Task.get_task(task_id=task_id)
    print(f"Input task id={task_id} artifacts {list(task.artifacts.keys())}".format())

    return task.artifacts

In [4]:
task_artifacts = get_task_artifacts(task_id=task_id)

Input task id=9e06dc7d106c405eb8fe3cdf55173244 artifacts ['data', 'feature_store', 'metadata']


In [16]:
import pandas as pd
import datetime as dt
data = task_artifacts["data"].get()

In [17]:
data.index = pd.to_datetime(data.index)

In [18]:
data.index.get_level_values('datetime_utc').max()

Timestamp('2023-06-30 20:00:00')

In [20]:
target = "energy_consumption"
forcast_horizon = data.index.get_level_values('datetime_utc').max() - dt.timedelta(hours=24*3)
forcast_horizon

Timestamp('2023-06-27 20:00:00')

In [21]:
import datetime as dt

# Split the data into training and testing sets
train_data = data.loc[data.index.get_level_values('datetime_utc') < forcast_horizon]
test_data = data.loc[data.index.get_level_values('datetime_utc') >= forcast_horizon]

# Display the shapes of the resulting DataFrames
print('Training data shape:', train_data.shape)
print('Testing data shape:', test_data.shape)

Training data shape: (158612, 18)
Testing data shape: (5548, 18)


In [22]:
y_train = train_data[target]
X_train = train_data.drop(target, axis=1)
y_test = test_data[target]
X_test = test_data.drop(target, axis=1)

print('Training labels shape:', y_train.shape)
print('Training features shape:', X_train.shape)
print('Testing labels shape:', y_test.shape)
print('Testing features shape:', X_test.shape)


Training labels shape: (158612,)
Training features shape: (158612, 17)
Testing labels shape: (5548,)
Testing features shape: (5548, 17)


In [23]:
model_args = {
    "base_score": 0.5,
    "booster": "gbtree",
    "n_estimators": 200,
    "early_stopping_rounds": 50,
    "objective": "reg:squarederror",
    "max_depth": 3,
    # "learning_rate": 0.01,
    "random_state": 42,
    "enable_categorical": True,
    "eval_metric": "rmse",
}

In [25]:
import xgboost as xgb


model = xgb.XGBRegressor(**model_args)
model.fit(
    X_train,
    y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    verbose=50,
)

/media/ubuntu_data/quantnh/anaconda3/envs/quan_env/lib/python3.11/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/media/ubuntu_data/quantnh/anaconda3/envs/quan_env/lib/python3.11/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/media/ubuntu_data/quantnh/anaconda3/envs/quan_env/lib/python3.11/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/media/ubuntu_data/quantnh/anaconda3/envs/quan_env/lib/python3.11/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be 

[0]	validation_0-rmse:91547.81110	validation_1-rmse:86146.08210


/media/ubuntu_data/quantnh/anaconda3/envs/quan_env/lib/python3.11/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/media/ubuntu_data/quantnh/anaconda3/envs/quan_env/lib/python3.11/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/media/ubuntu_data/quantnh/anaconda3/envs/quan_env/lib/python3.11/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/media/ubuntu_data/quantnh/anaconda3/envs/quan_env/lib/python3.11/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be 

[50]	validation_0-rmse:10650.78285	validation_1-rmse:4860.60669
[100]	validation_0-rmse:9660.75436	validation_1-rmse:4833.08949
[150]	validation_0-rmse:9117.99683	validation_1-rmse:4599.95235
[199]	validation_0-rmse:8683.93590	validation_1-rmse:4892.77372


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=50,
             enable_categorical=True, eval_metric='rmse', feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [26]:
PROJECT_NAME =  "cs2215-project"
from clearml import Task, TaskTypes

task = Task.init(
    project_name=PROJECT_NAME,
    task_name="Training model",
    task_type=TaskTypes.training,
    tags="training-pipeline",
)

ClearML Task: created new task id=1f5f3fcc1ecc4717865fc0a5d543b828
2023-10-14 19:47:12,988 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: http://100.113.148.58:8080/projects/50b14f4bf46b432b881b52b67d2b9e4d/experiments/1f5f3fcc1ecc4717865fc0a5d543b828/output/log


In [27]:
import xgboost as xgb


model = xgb.XGBRegressor(**model_args)
model.fit(
    X_train,
    y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    verbose=50,
)

/media/ubuntu_data/quantnh/anaconda3/envs/quan_env/lib/python3.11/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/media/ubuntu_data/quantnh/anaconda3/envs/quan_env/lib/python3.11/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/media/ubuntu_data/quantnh/anaconda3/envs/quan_env/lib/python3.11/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/media/ubuntu_data/quantnh/anaconda3/envs/quan_env/lib/python3.11/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be 

[0]	validation_0-rmse:91547.81110	validation_1-rmse:86146.08210
[50]	validation_0-rmse:10650.78285	validation_1-rmse:4860.60669
[100]	validation_0-rmse:9660.75436	validation_1-rmse:4833.08949
[150]	validation_0-rmse:9117.99683	validation_1-rmse:4599.95235
[199]	validation_0-rmse:8683.93590	validation_1-rmse:4892.77372


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=50,
             enable_categorical=True, eval_metric='rmse', feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [28]:
import joblib

In [29]:
joblib.dump(model, "../models/model.pkl", compress=True)
task.upload_artifact("model", "../models/model.pkl")

True

In [30]:
task.close()

In [31]:
model_task = Task.get_task(task_id="1f5f3fcc1ecc4717865fc0a5d543b828")
model_ckpt = model_task.artifacts["model"].get()
loaded_model = joblib.load(model_ckpt)

In [32]:
loaded_model

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=50,
             enable_categorical=True, eval_metric='rmse', feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)